<a href="https://colab.research.google.com/github/custom-hyper/Analytics/blob/main/DeFi_Lama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Setup

In [ ]:
#@title  Install  Packages { display-mode: "code" }
!pip install requests
!pip install json


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [ ]:
#@title Connect to Drive { display-mode: "code" }
 from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
#@title  Import Libraries { display-mode: "code" }
import requests
import json
from google.colab import data_table
import sqlite3 as sq
import pandas as pd
import logging
import pandas as pd
import logging
import time
from tqdm.notebook import tqdm
from datetime import datetime
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [43]:
#@title Connect to SQLite { display-mode: "code" }

path = r'/content/drive/MyDrive/workspace/pipelines/db/'
name = 'db.sqlite'

conn = sq.connect(path + name)


# Data Extraction

Please use  df['slug'] as primary key if loaded into SQL light

## Protocols ID List

In [63]:
#@title Call API { display-mode: "code" }



#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
DeFi = json.loads(output)
full_talbe = pd.json_normalize(DeFi)
full_talbe['id'] = full_talbe['symbol']  + '.' + full_talbe['name']
full_talbe['id'].replace(' ','')
full_talbe['Notes'] = ''

#UNIX timestamp: created
full_talbe['UNIX'] = int( time.time())



full_talbe.head(1)

,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,chainTvls.Metis-pool2,chainTvls.Harmony-Treasury,chainTvls.Metis-Treasury,chainTvls.Velas-staking,chainTvls.Celo-Treasury,chainTvls.Proton-staking,chainTvls.Proton-borrowed,chainTvls.Ergo-staking,Notes,UNIX
0,CRV.Curve,Curve,0xD533a949740bb3306d119CC777fa900bA034cd52,CRV,https://curve.fi,Curve is a decentralized exchange liquidity po...,Multi-Chain,https://icons.llama.fi/curve.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1649819715


In [65]:
#@title Save To Directory{ display-mode: "code" }
sfile = "deFi_lama_id"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
full_talbe.to_csv(path + sfile)

In [67]:
#@title Load to SQLite { display-mode: "code" }

table = sfile

full_talbe.to_sql(table, conn , if_exists='replace', index=False) # writes to file

 # Test Connection
df = pd.read_sql('select * from {}'.format(table), conn)

df.head()

InterfaceError: ignored

In [68]:
#@title Show Stats { display-mode: "code" }
full_talbe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482 entries, 0 to 1481
Columns: 300 entries, id to UNIX
dtypes: float64(274), int64(1), object(25)
memory usage: 3.4+ MB


 ## TVL Timeseries

In [32]:
#@title Call API  { display-mode: "code" }

#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
table = json.loads(output)
df = pd.json_normalize(table)
slug = df['slug'][:3]

path = "https://api.llama.fi/protocol/"
frames = []


try:
  for i in tqdm(slug):
    response = requests.get(path + i)
    output = response.text
    DeFi = json.loads(output)

    full_table = pd.json_normalize(DeFi['tvl']) 
    full_table['slug'] = i
    

    #results = [full_table]
    frames.append(full_table)

    #Slow down requests
    
    time.sleep(60/50)
    #Status report
    
    
except ConnectionError:
    time.sleep(60)


single = pd.concat(frames) #Careful, this line must be outside of the for loop
#single['date_Standard']  = datetime.fromtimestamp(time).strftime('%Y-%m-%d')


single.head()

  


NameError: ignored

In [58]:
#@title Save To Directory{ display-mode: "code" }
sfile = "deFi_lama_tvl"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
single.to_csv(path + sfile)

In [59]:
#@title Load to SQLite { display-mode: "code" }

table = "deFi_lama_tvl"

single.to_sql(table, conn , if_exists='replace', index=False) # writes to file

 # Test Connection
df = pd.read_sql('select * from {}'.format(table), conn)

df.head()


,date,totalLiquidityUSD,slug
0,1581206400,1.163733e+06,curve
1,1581292800,1.372110e+06,curve
2,1581379200,9.607534e+05,curve
3,1581465600,1.006408e+06,curve
4,1581552000,1.258776e+06,curve


In [28]:
#@title Show Stats { display-mode: "code" }
single.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1632 entries, 0 to 356
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               1632 non-null   int64  
 1   totalLiquidityUSD  1632 non-null   float64
 2   slug               1632 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 51.0+ KB


In [23]:
conn.close()

## Protocol General Information Data

In [31]:
#@title Call API  { display-mode: "code" }


#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
DeFi = json.loads(output)
full_talbe = pd.json_normalize(DeFi)
full_talbe['id'] = full_talbe['symbol']  + '.' + full_talbe['name']
full_talbe['id'].replace(' ','')
full_talbe['Notes'] = ''

#UNIX timestamp: created
full_talbe['UNIX'] = int( time.time())



full_talbe.head(1)



,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,chainTvls.zkSync,chainTvls.zkSync-offers,chainTvls.Metis-pool2,chainTvls.Harmony-Treasury,chainTvls.Metis-Treasury,chainTvls.Velas-staking,chainTvls.Celo-Treasury,chainTvls.Ergo-staking,Notes,UNIX
0,CRV.Curve,Curve,0xD533a949740bb3306d119CC777fa900bA034cd52,CRV,https://curve.fi,Curve is a decentralized exchange liquidity po...,Multi-Chain,https://icons.llama.fi/curve.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1649818205


In [60]:
 #@title Save To Directory{ display-mode: "code" }
 
table = "deFi_lama_general"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
full_talbe.to_csv(path+ table)

In [50]:
@title Dataset Stats { display-mode: "code" }
full_talbe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482 entries, 0 to 1481
Columns: 300 entries, id to UNIX
dtypes: float64(274), int64(1), object(25)
memory usage: 3.4+ MB


In [61]:
#@title Load to SQLite { display-mode: "code" }

full_talbe.to_sql(table, conn , if_exists='replace', index=False) # writes to file

 # Test Connection
df = pd.read_sql('select * from {}'.format(table), conn)

df.head()

InterfaceError: ignored

In [49]:
full_talbe.describe()

,tvl,change_1h,change_1d,change_7d,staking,fdv,mcap,chainTvls.Optimism,chainTvls.Avalanche,chainTvls.Harmony,...,chainTvls.Arbitrum-offers,chainTvls.zkSync,chainTvls.zkSync-offers,chainTvls.Metis-pool2,chainTvls.Harmony-Treasury,chainTvls.Metis-Treasury,chainTvls.Velas-staking,chainTvls.Celo-Treasury,chainTvls.Ergo-staking,UNIX
count,1.482000e+03,1292.000000,1267.000000,1250.000000,6.770000e+02,6.300000e+02,1.037000e+03,3.100000e+01,1.920000e+02,5.000000e+01,...,1.0,1.0,1.000000,3.000000e+00,1.000000,1.000000,2.000000e+00,1.0,1.000000e+00,1.482000e+03
mean,1.682046e+08,-0.068228,16.302724,565.950765,3.231039e+07,1.206910e+12,9.640125e+07,1.445945e+07,5.715993e+07,1.437001e+07,...,1.0,0.0,314672.090376,1.165436e+06,0.000479,1649.659486,5.946350e+06,0.0,8.657726e+06,1.649818e+09
std,1.143884e+09,9.722319,343.729155,18222.868136,3.024581e+08,2.811855e+13,6.229204e+08,3.218974e+07,2.545235e+08,4.257735e+07,...,NaN,NaN,NaN,1.982012e+06,NaN,NaN,7.369549e+06,NaN,NaN,0.000000e+00
min,0.000000e+00,-100.000000,-100.000000,-100.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.0,0.0,314672.090376,1.199671e+04,0.000479,1649.659486,7.352925e+05,0.0,8.657726e+06,1.649818e+09
25%,2.198746e+04,-0.214717,0.000000,-13.923172,2.974539e+03,2.196857e+06,0.000000e+00,1.220315e+05,3.147269e+02,3.245448e+04,...,1.0,0.0,314672.090376,2.113247e+04,0.000479,1649.659486,3.340822e+06,0.0,8.657726e+06,1.649818e+09
50%,1.264605e+06,-0.000003,0.948617,-8.452990,2.125460e+05,2.660364e+07,6.899290e+05,1.437414e+06,2.013976e+05,2.126428e+05,...,1.0,0.0,314672.090376,3.026822e+04,0.000479,1649.659486,5.946350e+06,0.0,8.657726e+06,1.649818e+09
75%,1.917017e+07,0.100000,2.374985,-0.960578,2.852915e+06,1.404797e+08,1.431818e+07,1.027160e+07,3.060992e+06,5.155996e+06,...,1.0,0.0,314672.090376,1.742155e+06,0.000479,1649.659486,8.551879e+06,0.0,8.657726e+06,1.649818e+09
max,2.020057e+10,291.448423,9840.839377,642640.859062,7.292257e+09,7.043142e+14,1.100408e+10,1.678286e+08,2.653505e+09,2.266821e+08,...,1.0,0.0,314672.090376,3.454042e+06,0.000479,1649.659486,1.115741e+07,0.0,8.657726e+06,1.649818e+09


In [ ]:
#@title Dataset Stats { display-mode: "code" }
full_talbe.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 297 columns):
 #    Column                            Dtype  
---   ------                            -----  
 0    id                                object 
 1    name                              object 
 2    address                           object 
 3    symbol                            object 
 4    url                               object 
 5    description                       object 
 6    chain                             object 
 7    logo                              object 
 8    audits                            object 
 9    audit_note                        object 
 10   gecko_id                          object 
 11   cmcId                             object 
 12   category                          object 
 13   chains                            object 
 14   module                            object 
 15   twitter                           object 
 16   audit_links           

 ## Total Combined TVL

In [55]:
  #@title API call { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table

import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt

path = "https://api.llama.fi/charts"
#slug = 'aave' 
#General Information table
response = requests.get(path)
output = response.text
DeFi = json.loads(output)
totalLiq_talbe = pd.json_normalize(DeFi)
totalLiq_talbe.head()



,date,totalLiquidityUSD
0,1541203200,34684.037747
1,1541289600,41292.993049
2,1541376000,41550.309965
3,1541462400,43233.560074
4,1541548800,153862.566498


In [56]:
 #@title Save To DeFi Lama Dataset Directory{ display-mode: "code" }
 
table = "deFi_lama_total_TVL"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
totalLiq_talbe.to_csv(path+ table)

In [57]:
#@title Load General Information Data to SQLite { display-mode: "code" }


totalLiq_talbe.to_sql(table, conn , if_exists='replace', index=False) # writes to file

 # Test Connection
df = pd.read_sql('select * from {}'.format(table), conn)

df.head()

,date,totalLiquidityUSD
0,1541203200,34684.037747
1,1541289600,41292.993049
2,1541376000,41550.309965
3,1541462400,43233.560074
4,1541548800,153862.566498


# Feature Engineering